In [18]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from scipy.sparse import csc_matrix
import time
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [19]:
train_fname = '../data/train_cat_1.csv'
test_fname = '../data/test_cat_1.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
n_samples,n_variables = df.shape

In [20]:
f_cat = ['VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'FIRST_CLASSE','TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']

In [21]:
y_fname = '../data/y_train_1.csv'
df_y = pd.read_csv(y_fname, sep=';')

In [22]:
X_train = df[f_cat].values
X_test = df_test[f_cat].values
y_train = df_y['VARIABLE_CIBLE'].values == 'GRANTED'

In [23]:
ohencoder = OneHotEncoder()
X_train = ohencoder.fit_transform(X_train)
X_test = ohencoder.transform(X_test)

In [24]:
X_train = csc_matrix(X_train)
X_test = csc_matrix(X_test)

In [18]:
n_taken = n_samples
clf = RandomForestClassifier(max_depth=30)
start=time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print(time.time() - start)
y_pred_train = clf.predict_proba(X_train)[:, 1]
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

36.7774038315
Score (optimiste) sur le train : 0.678383736387


In [19]:
# 0.63438457837 max_depth=12
# 0.639923239977 max_depth=15
# 0.650262261772 max_depth=20
# 0.678383736387 max_depth=30 en 36.7774038315 secondes lol

In [13]:
n_taken = n_samples
nb_thread = 4
clf = RandomForestClassifier()
# parameters = {'max_depth':[13,14,15],'oob_score':[False,True],'min_samples_leaf':[4,5,6],'min_weight_fraction_leaf':[0,0.1],'min_samples_split':[1,2,3]}
parameters = {'n_estimators':[500],'max_depth':[90],'oob_score':[False],'min_samples_leaf':[2],'min_samples_split':[5]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train[0:n_taken],y_train[0:n_taken])
print(time.time() - start)
#y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
print('Score raisonnable de la CV : {}'.format(gscv.best_score_))
#print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(gscv.best_params_)

In [ ]:
# A battre : 0.669496310911 (RF sans OHE)

# 0.634285098531 {'max_depth': 20}
# 0.64671837006 {'max_depth': 50}
# 0.651821500881 {'max_depth': 80}
# 0.651913926046 {'max_depth': 90}
# 0.657541811484 {'oob_score': False, 'max_depth': 90, 'min_samples_leaf': 2}
# 0.65800847239 {'min_samples_split': 3, 'oob_score': False, 'max_depth': 90, 'min_samples_leaf': 2}
# 0.658067765011 {'min_samples_split': 5, 'oob_score': False, 'max_depth': 90, 'min_samples_leaf': 2}
# 0.662391108867 {'min_samples_split': 6, 'n_estimators': 30, 'oob_score': False, 'max_depth': 90, 'min_samples_leaf': 2}
# 0.664345066245 {'min_samples_split': 5, 'n_estimators': 500, 'oob_score': False, 'max_depth': 90, 'min_samples_leaf': 2}
# En 2560.25390887 secondes ! De quoi être inquiet...